In [135]:
import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import (Activation, Concatenate, Conv2D,
                                     Conv2DTranspose, Input, LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow_addons.layers import InstanceNormalization

import tensorflow_datasets as tfds

%matplotlib inline

In [136]:
generator_g = tf.keras.models.load_model('generator_g.h5')
generator_f = tf.keras.models.load_model('generator_f.h5')
discriminator_x = tf.keras.models.load_model('discriminator_x.h5')
discriminator_y = tf.keras.models.load_model('discriminator_y.h5')


In [137]:
# Load dataset
data, metadata = tfds.load('cycle_gan/monet2photo', with_info=True, as_supervised=True)

train_x, train_y, test_x, test_y = data['trainA'], data['trainB'], data['testA'], data['testB']

In [138]:
next(iter(train_x))

(<tf.Tensor: shape=(256, 256, 3), dtype=uint8, numpy=
 array([[[111, 133, 146],
         [109, 131, 144],
         [105, 126, 143],
         ...,
         [ 51,  46,  50],
         [ 44,  42,  45],
         [ 78,  76,  79]],
 
        [[112, 134, 148],
         [106, 127, 144],
         [ 99, 120, 139],
         ...,
         [ 44,  39,  43],
         [ 40,  38,  41],
         [ 64,  62,  65]],
 
        [[113, 134, 155],
         [105, 126, 147],
         [ 94, 114, 138],
         ...,
         [ 37,  32,  36],
         [ 37,  35,  38],
         [ 46,  44,  47]],
 
        ...,
 
        [[ 31,  21,  20],
         [ 20,  12,  10],
         [ 14,   4,   3],
         ...,
         [ 80,  84,  67],
         [ 72,  79,  61],
         [ 95, 104,  85]],
 
        [[ 37,  25,  25],
         [ 28,  18,  17],
         [ 24,  12,  12],
         ...,
         [ 65,  69,  52],
         [ 60,  67,  51],
         [ 88,  97,  80]],
 
        [[ 49,  37,  37],
         [ 46,  34,  34],
         [ 45,

In [114]:
data

{'trainA': <PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>,
 'trainB': <PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>,
 'testA': <PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>,
 'testB': <PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>}

AttributeError: 'MapDataset' object has no attribute 'makeOneShotIterator'

In [116]:
# Settings
epochs = 50

LAMBDA = 10

img_rows, img_cols, channels = 256, 256, 3
weight_initializer = RandomNormal(stddev=0.02)

gen_g_optimizer = gen_f_optimizer = Adam(lr=0.0002, beta_1=0.5)
dis_x_optimizer = dis_y_optimizer = Adam(lr=0.0002, beta_1=0.5)

/Users/billyluqiu/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [117]:
# Normalize images to [-1, 1] and reshape
def preprocess_image(image, _):
    return tf.reshape(tf.cast(tf.image.resize(image, (int(img_rows), int(img_cols))), tf.float32) / 127.5 - 1, (1, img_rows, img_cols, channels))

In [118]:
# Map the normalization onto the dataset
train_x = train_x.map(preprocess_image)
train_y = train_y.map(preprocess_image)
test_x = test_x.map(preprocess_image)
test_y = test_y.map(preprocess_image)

In [119]:
test_x

<MapDataset shapes: (1, 256, 256, 3), types: tf.float32>

In [120]:
def generate_images():
    # Sample images
    print(test_x)
    x = next(iter(test_x.shuffle(1000))).numpy()
    y = next(iter(test_y.shuffle(1000))).numpy()
    print(x)
    # Get predictions for those images
    y_hat = generator_g.predict(x.reshape((1, img_rows, img_cols, channels)))
    x_hat = generator_f.predict(y.reshape((1, img_rows, img_cols, channels)))
    


In [121]:
generate_images()

<MapDataset shapes: (1, 256, 256, 3), types: tf.float32>
[[[[-0.56078434 -0.23137254 -0.41960782]
   [-0.5686275  -0.27058822 -0.4352941 ]
   [-0.7019608  -0.41960782 -0.5294118 ]
   ...
   [-0.20784312  0.12156868 -0.64705884]
   [-0.12156862  0.27058828 -0.41176468]
   [-0.19215685  0.05882359 -0.5294118 ]]

  [[-0.5058824  -0.17647058 -0.3490196 ]
   [-0.5137255  -0.2235294  -0.36470586]
   [-0.62352943 -0.35686272 -0.46666664]
   ...
   [-0.23921567  0.10588241 -0.6392157 ]
   [-0.32549018  0.09803927 -0.5764706 ]
   [-0.20784312  0.05882359 -0.52156866]]

  [[-0.5764706  -0.23921567 -0.38823527]
   [-0.5686275  -0.27058822 -0.38823527]
   [-0.62352943 -0.36470586 -0.45098037]
   ...
   [-0.23137254  0.15294123 -0.5686275 ]
   [-0.372549    0.05882359 -0.5921569 ]
   [-0.11372548  0.1686275  -0.3960784 ]]

  ...

  [[-0.372549   -0.00392157 -0.3490196 ]
   [-0.5686275  -0.1607843  -0.4588235 ]
   [-0.6313726  -0.17647058 -0.49019605]
   ...
   [-0.827451   -0.5529412  -0.6       ]


2022-12-03 22:46:23.990310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-03 22:46:24.845119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


#BELOW THIS IS THE TEST DATA FROM KAGGLE

In [160]:
images = tf.keras.utils.image_dataset_from_directory(
  "/Users/billyluqiu/Desktop/fall2022/ece588Final/billy/images", batch_size=1)

Found 300 files belonging to 1 classes.


In [161]:
images2 = tf.keras.utils.image_dataset_from_directory(
  "/Users/billyluqiu/Desktop/fall2022/ece588Final/billy/images2", batch_size=1)

Found 7038 files belonging to 1 classes.


In [164]:
test_y = images.map(preprocess_image)

In [165]:
test_x = images2.map(preprocess_image)

In [166]:
test_x

<MapDataset shapes: (1, 256, 256, 3), types: tf.float32>

In [182]:
import matplotlib.image

In [192]:
i = 1
for x in test_x:
    x_numpy = x.numpy()
    y_hat = generator_f.predict(x_numpy.reshape((1, img_rows, img_cols, channels)))
    normalizedData = (y_hat[0]-np.min(y_hat[0]))/(np.max(y_hat[0])-np.min(y_hat[0]))

    matplotlib.image.imsave('test_images/'+ str(i) + '.png', normalizedData)
    i += 1
    

2022-12-03 23:17:36.955437: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [187]:
np.amin(y_hat)

-0.99967366